In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPool2D())
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense (Dense)               (None, 10)                5

In [2]:
from tokenize import Double
import torch

nn = torch.nn
F = torch.nn.functional

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)    # in ch, out ch, kernel size, stride=1, padding=0
        self.conv2 = nn.Conv2d(32, 16, 3, padding=0)
        self.fc1 = nn.Linear(16 * 6 * 6, 10)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.conv1(x)
        print(f'conv1(x)=({x.shape})')   # x(1,3,32,32) > conv > x(1,32,30,30)
        x = self.pool(F.relu(x))        # x > pool > x(1,32,15,15)
        print(f'pool(x)=({x.shape})')
        x = self.pool(F.relu(self.conv2(x)))    # (1,16,13,13) > (1,16,6,6)
        print(f'pool(conv2)=({x.shape})')   # torch.Size([1,16,6,6])
        x = x.view(-1, 16 * 6 * 6)
        x = F.log_softmax(self.fc1(x), dim=-1)
        print(f'log_softmax={x}, ({x.shape})')
        return x
        
model = Net()
# x = torch.rand(1, 3, 32, 32)
x = torch.arange(3 * 32 * 32)
x = x.reshape(1, 3, 32, 32).float()
print(x.shape)
# print(x)

yhat = model.forward(x)
print(model)
print(yhat.shape)
print(yhat)

torch.Size([1, 3, 32, 32])
conv1(x)=(torch.Size([1, 32, 30, 30]))
pool(x)=(torch.Size([1, 32, 15, 15]))
pool(conv2)=(torch.Size([1, 16, 6, 6]))
log_softmax=tensor([[ -82.1605, -759.8065, -445.3680, -367.8447, -369.8665, -160.6333,
         -402.4622, -200.7516,    0.0000, -370.9247]],
       grad_fn=<LogSoftmaxBackward0>), (torch.Size([1, 10]))
Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([1, 10])
tensor([[ -82.1605, -759.8065, -445.3680, -367.8447, -369.8665, -160.6333,
         -402.4622, -200.7516,    0.0000, -370.9247]],
       grad_fn=<LogSoftmaxBackward0>)
